In [1]:
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
from scipy.sparse.linalg import svds
from tqdm import tqdm
import re
import xml
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import pickle

In [2]:
def clean_text(text):
    '''Make text lowercase,remove punctuation
    .'''
    text = str(text).lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text

# Load datasets

In [3]:
questions =  pd.read_csv("input/questions.csv")
professionals = pd.read_csv("input/professionals.csv")
answers = pd.read_csv("input/answers.csv")

In [4]:
questions.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [5]:
professionals.head()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,2011-10-05 20:35:19 UTC+0000
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,2011-10-05 20:49:21 UTC+0000
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",NaN,NaN,2011-11-09 20:39:29 UTC+0000
4,e2d57e5041a44f489288397c9904c2b2,NaN,NaN,NaN,2011-12-10 22:14:44 UTC+0000


In [6]:
prof_ans = pd.merge(professionals, answers, how = 'left' ,
                    left_on = 'professionals_id', right_on = 'answers_author_id')
prof_ans_q = pd.merge(prof_ans, questions, how = 'left' ,
                      left_on = 'answers_question_id', right_on = 'questions_id')

In [7]:
prof_ans_q.head()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaN,NaN,2011-10-05 20:35:19 UTC+0000,7640a6e5d5224c8681cc58de860858f4,9ced4ce7519049c0944147afb75a8ce3,f6b9ca94aed04ba28256492708e74f60,2011-10-05 20:42:09 UTC+0000,<p>Basically three things: </p>\n<ol>\n<li>Big...,f6b9ca94aed04ba28256492708e74f60,05444a2f42454327b2ac4b463c0adbe0,2011-09-27 15:26:19 UTC+0000,What do top tier consulting firms look for in ...,Please explain the factors consulting firms lo...
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaN,NaN,2011-10-05 20:49:21 UTC+0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000,f18d7fca363d4b21a81e6683c5a86b96,0c673e046d824ec0ad0ebe012a0673e4,e214acfbe6644d65b889a3268828db9d,2012-10-01 04:35:42 UTC+0000,"<html><head></head><body><p>Hi Deja,</p>\n<p>K...",e214acfbe6644d65b889a3268828db9d,16908136951a48ed942738822cedd5c2,2012-09-09 05:33:25 UTC+0000,what does it take to be an anesthesiologist?,I am a sophomore who is interested in learning...
3,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000,5d670d5f8700402ab56bae609b06d02d,0c673e046d824ec0ad0ebe012a0673e4,9d211b99e17c46fbbaca03dc6b43f1c4,2012-10-01 04:55:03 UTC+0000,<html><head></head><body><p>It looks like this...,9d211b99e17c46fbbaca03dc6b43f1c4,16908136951a48ed942738822cedd5c2,2012-09-09 00:43:18 UTC+0000,what are some of the ups and downs of being a ...,I am a sophomore who is interested in learning...
4,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",NaN,NaN,2011-10-18 17:31:26 UTC+0000,6d823a750e294c75b10fafbbbaf19855,0c673e046d824ec0ad0ebe012a0673e4,e978b437413048c183e3cb556f90a878,2012-02-13 16:44:10 UTC+0000,<p><em>[Posted on behalf of a CareerVillage Pr...,e978b437413048c183e3cb556f90a878,177f482adc1a449284471f7c556ee8f0,2012-02-13 13:45:52 UTC+0000,What do I have to do to become an investor?,<p>I would like to invest in other companies w...


In [8]:
prof_ans_q = prof_ans_q[(~prof_ans_q["questions_title"].isna()) | (~prof_ans_q["questions_body"].isna()) ]

In [9]:
len(prof_ans_q)

50106

In [10]:
q = prof_ans_q["questions_title"] + " " + prof_ans_q["questions_body"]
q  = q.apply(lambda x:clean_text(x))
len(q)

50106

# Load Spacy

In [11]:
nlp = spacy.load('en_core_web_lg')

In [12]:
with nlp.disable_pipes():
    spacy_questions = [nlp(q1).vector for q1 in q ]

In [13]:
spacy_questions[0].shape

(300,)

In [14]:
pickle.dump(spacy_questions,open("spacy_questions.pkl","wb"))

# New Question

In [15]:
q_new = "I want to be a data scientist. What should I study"
with nlp.disable_pipes():
    q_new = [nlp(q_new).vector]


with open('spacy_questions.pkl', 'rb') as f:
    spacy_questions = pickle.load(f)

In [16]:
cosine_similarity(q_new,spacy_questions).shape

(1, 50106)

In [17]:
result = cosine_similarity(q_new,spacy_questions)
result.argmax()

48865

In [28]:
np.argsort(result)

array([[24362, 16008, 23445, ..., 18047, 48865, 49181]], dtype=int64)

In [18]:
np.argsort(result)[:,-10:]

array([[24223, 16339, 30993, 30968, 47722, 33136, 16221, 18047, 48865,
        49181]], dtype=int64)

In [30]:
np.argsort(result)[:,-10:][0][-1],np.argsort(result)[:,-10:][0][-2],np.argsort(result)[:,-10:][0][-3],

(49181, 48865, 18047)

In [37]:
result[:,49181],result[:,48865],result[:,18047]

(array([0.9815354], dtype=float32),
 array([0.9815354], dtype=float32),
 array([0.9607585], dtype=float32))

In [19]:
result.shape

(1, 50106)

In [20]:
q.iloc[result.argmax()] , result[:,result.argmax()]

('what should i learn to be a data scientist i want to be a data scientist  what online courses should i take  datascience\r',
 array([0.9815354], dtype=float32))

In [22]:
prof_ans_q.iloc[np.argsort(result)[:,-10:][0][-1]]["answers_body"]

'<p>You should search for Algorithm videos. Usually when studying data, you would need to know about databases structure, analytics skills, and some other logics. Another thing you could do would be start analyzing some small real cases like how long does it take to go from your house to the supermarket and what you could do to reduce the time? or how often do you drink water (time gap between each occurence). How could you track that? and how could you improve it? is it good?</p><p>these are a few examples on how you could analyze stuff.</p>'

In [23]:
prof_ans_q.iloc[np.argsort(result)[:,-10:][0][-2]]["answers_body"]

'<p> </p><p>Hello Chong G.</p><p> </p><p>I am not a data scientist, but I think I can give you some advice on this. Nowadays, an increasing number of professions are requiring analytics capabilities.</p><p> </p><p> </p><p>There are some core things you should learn to handle great amount of data, like:</p><p>&nbsp;</p><p>Relational Database concepts;</p><p>SQL - Computer language for creating and managing databases;</p><p>Excel;</p><p>Programing languages such as C, VBA, R...&nbsp;</p><p><br></p><p>You should also consider learning how to display the data in an organized way and Power BI / Think-Cell are great for that</p><p>  </p><p>There are several tutorials around the internet about those topics and also focused courses. I personally recommend the latter, because it is easier to progress through the topics.</p><p><br></p><p>Hope my advice was helpful to you!</p><p> </p>'

In [24]:
prof_ans_q.iloc[np.argsort(result)[:,-10:][0][-3]]["answers_body"]

'<p>It is wonderful that you want to learn software coding and become a software engineer. This is  certainly a field that has and will continue to have lots of exciting prospects for learning and growth. One of the ways to become a software engineer is to get a degree in Computer Science or Information technology or computer applications. For applying to a college to study these disciplines one must first of all be interested in and study Maths as well as Physics and Chemistry (so called PCM subjects) till 12th standard. Meanwhile, in many schools, computers is taught as a subject in 9th, 10th, 11th and 12th standards. As part of the computer subject you will learn about programming and learn the basics of coding in a language such as Java. If your school/PUC college does not offer computers as a subject then you can enroll for a programming class outside school or if you have access to a computer you can learn programming on your own. Based on the above if you can give me more detail